<table  align="left" width="100%"> <tr>
        <td  style="background-color:#ffffff;"><a href="https://qworld.net" target="_blank"><img src="../images/qworld.jpg" width="35%" align="left"></a></td>
        <td  align="right" style="background-color:#ffffff;vertical-align:bottom;horizontal-align:right">
            prepared by <a href="https://iitis.pl/pl/person/aglos" target="_blank"  >Adam Glos</a>
        </td>        
</tr></table>

<table width="100%"><tr><td style="color:#bbbbbb;background-color:#ffffff;font-size:11px;font-style:italic;text-align:right;">This cell contains some macros. If there is a problem with displaying mathematical formulas, please run this cell to load these macros. </td></tr></table>

$ \renewcommand{\bra}[1]{\langle #1|} $
$ \renewcommand{\ket}[1]{|#1\rangle} $
$ \renewcommand{\braket}[2]{\langle #1|#2\rangle} $
$ \renewcommand{\dot}[2]{ #1 \cdot #2} $
$ \renewcommand{\biginner}[2]{\left\langle #1,#2\right\rangle} $
$ \renewcommand{\mymatrix}[2]{\left( \begin{array}{#1} #2\end{array} \right)} $
$ \renewcommand{\myvector}[1]{\mymatrix{c}{#1}} $
$ \renewcommand{\myrvector}[1]{\mymatrix{r}{#1}} $
$ \renewcommand{\mypar}[1]{\left( #1 \right)} $
$ \renewcommand{\mybigpar}[1]{ \Big( #1 \Big)} $
$ \renewcommand{\sqrttwo}{\frac{1}{\sqrt{2}}} $
$ \renewcommand{\dsqrttwo}{\dfrac{1}{\sqrt{2}}} $
$ \renewcommand{\onehalf}{\frac{1}{2}} $
$ \renewcommand{\donehalf}{\dfrac{1}{2}} $
$ \renewcommand{\hadamard}{ \mymatrix{rr}{ \sqrttwo & \sqrttwo \\ \sqrttwo & -\sqrttwo }} $
$ \renewcommand{\vzero}{\myvector{1\\0}} $
$ \renewcommand{\vone}{\myvector{0\\1}} $
$ \renewcommand{\vhadamardzero}{\myvector{ \sqrttwo \\  \sqrttwo } } $
$ \renewcommand{\vhadamardone}{ \myrvector{ \sqrttwo \\ -\sqrttwo } } $
$ \renewcommand{\myarray}[2]{ \begin{array}{#1}#2\end{array}} $
$ \renewcommand{\X}{ \mymatrix{cc}{0 & 1 \\ 1 & 0}  } $
$ \renewcommand{\Z}{ \mymatrix{rr}{1 & 0 \\ 0 & -1}  } $
$ \renewcommand{\Htwo}{ \mymatrix{rrrr}{ \frac{1}{2} & \frac{1}{2} & \frac{1}{2} & \frac{1}{2} \\ \frac{1}{2} & -\frac{1}{2} & \frac{1}{2} & -\frac{1}{2} \\ \frac{1}{2} & \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} \\ \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} & \frac{1}{2} } } $
$ \renewcommand{\CNOT}{ \mymatrix{cccc}{1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0} } $
$ \renewcommand{\norm}[1]{ \left\lVert #1 \right\rVert } $
$ \renewcommand{\pstate}[1]{ \lceil #1  \rfloor } $

# Grover's Algorithm revisited

Grover's Search is one of the well-known quantum algorithms. Given an oracle which marks some of the elements in the search space, Grover's Algrorithm outputs the marked elements with high probability. This notebook only serves as a recap for Grover's Search and presents the code in Cirq. We refer to Bronze for the actual explanation of the algorithm.

The quantum circuit for the algorithm takes the form:

<img src="../images/grover.png" width="60%" align="center">

Oracle is responsible for marking the elements, while Grover diffusion operator amplifies the probability of measuring the marked elements. They are applied interchangeably for $\sqrt{N/k}$ times, where $N$ is the size of the searh space, and $k$ is the number of marked elements.

## Initialization 

The Grover's search starts with creating an equal superposition on all qubits. We start with implementing this step of the algorithm.

### Task 1 
Create a 4-qubit circuit named `circuit` and apply a Hadamard gate on each qubit. Convince yourself about the correctness of the circuit by displaying it. Print the quantum state with `Simulator` without measurement.

In [2]:
import cirq
from cirq import H

#
# your solution
#
qlist = cirq.LineQubit.range(4)
circuit = cirq.Circuit()

circuit.append(H.on_each(*qlist))

print(circuit)

# to print the state
s = cirq.Simulator()
print(s.simulate(circuit))
print(s.simulate(circuit).dirac_notation())

0: ───H───

1: ───H───

2: ───H───

3: ───H───
measurements: (no measurements)
output vector: [0.24999997+0.j 0.24999997+0.j 0.24999997+0.j 0.24999997+0.j
 0.24999997+0.j 0.24999997+0.j 0.24999997+0.j 0.24999997+0.j
 0.24999997+0.j 0.24999997+0.j 0.24999997+0.j 0.24999997+0.j
 0.24999997+0.j 0.24999997+0.j 0.24999997+0.j 0.24999997+0.j]
0.25|0000⟩ + 0.25|0001⟩ + 0.25|0010⟩ + 0.25|0011⟩ + 0.25|0100⟩ + 0.25|0101⟩ + 0.25|0110⟩ + 0.25|0111⟩ + 0.25|1000⟩ + 0.25|1001⟩ + 0.25|1010⟩ + 0.25|1011⟩ + 0.25|1100⟩ + 0.25|1101⟩ + 0.25|1110⟩ + 0.25|1111⟩


[click for our solution](B01_Grover_Algorithm_Revisited_Solutions.ipynb#task1)

## A simple oracle

Here we will design a simple oracle, which marks the state $\ket{1111}$. Here marking means to change the sign of state $\ket{1111}$ to $-\ket{1111}$, while leaving the other states unchanged. We know that the operator

$$ Z = \begin{bmatrix} 1 & 0 \\ 0 & -1 \end{bmatrix}$$

changes the state $\ket{1}$ into $-\ket{1}$. Even though stand-alone $Z$ gate applied to the third qubit changes the state $\ket{1111}$ into -$\ket{1111}$, it also changes, for example, the state $\ket{0011}$ into -$\ket{1011}$. On the other hand, we can design a multi-controlled $Z$ gate for our purpose: $Z$ gate is activated on the third qubit when the other qubits are in states $\ket{1}$, i.e., $b_0=b_1=b_2 = 1$.


Now we show how to implement this multiple-controlled gate. In order to apply $Z$ gate on the third qubit, we use `cirq.Z(qq[3])` and we control this gate with control on the remaining qubits with `cirq.Z(qq[3]).controlled_by(qq[0], qq[1], qq[2])`.  

In [3]:
import cirq

qq = cirq.LineQubit.range(4)
circuit = cirq.Circuit()

CCCZ = cirq.Z(qq[3]).controlled_by(qq[0], qq[1], qq[2])
# alternative version:
# CCCZ = cirq.Z(qq[3]).controlled_by(*(qq[0:3]))
circuit.append(CCCZ)

print(circuit)

0: ───@───
      │
1: ───@───
      │
2: ───@───
      │
3: ───Z───


#### Multi-controlled gates in general

Alternatively, we can define a multi-controlled gate, and specify the target and control qubits later. This will be particularly useful, when the same gate will be used many times on a various collection of qubits.

In [4]:
import cirq

qq = cirq.LineQubit.range(4)
circuit = cirq.Circuit()

CCCZ = cirq.Z.controlled(3)
CCCX = cirq.X.controlled(3)

circuit.append(CCCZ(*qq))
circuit.append(CCCX(*qq))
circuit.append(CCCX(qq[0], qq[1], qq[3], qq[2]))

print(circuit)

0: ───@───@───@───
      │   │   │
1: ───@───@───@───
      │   │   │
2: ───@───@───X───
      │   │   │
3: ───@───X───@───


Note that for `CCCZ`, the target qubit is not distinguished when displayed. This is because controlled $Z$ operator acts exactly the same independent of the chosen target qubit. This can be easily shown as `CCCZ` acts nontrivially (in case of basis states) only on $\ket{1111}$ -- as you can see there is no way to distinguish which qubit is a target one from this form.

Note that a multi-controlled operator is not the same as several single-qubits controls. To see this, let us simulate the operators for both cases. To do so, we will compare the unitary matrices. To make matrices smaller and easier to compare, we will only compare 3-qubits operations.

In [5]:
import cirq
from cirq import Z

# z gate controlled by qubits 0 and 1
qq = cirq.LineQubit.range(3)
circ_multi = cirq.Circuit()
circ_multi.append(Z(qq[2]).controlled_by(qq[0], qq[1]))

qq = cirq.LineQubit.range(3)
circ_single = cirq.Circuit()
# z gate controlled by qubit 0
circ_single.append(Z(qq[2]).controlled_by(qq[0]))
# z gate controlled by qubit 1
circ_single.append(Z(qq[2]).controlled_by(qq[1]))

print(cirq.unitary(circ_multi))
print()
print(cirq.unitary(circ_single))

[[ 1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  1.+0.j  0.+0.j]
 [-0.+0.j -0.+0.j -0.+0.j -0.+0.j -0.+0.j -0.+0.j -0.+0.j -1.+0.j]]

[[ 1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [-0.+0.j -0.+0.j -0.+0.j -1.+0.j -0.+0.j -0.+0.j -0.+0.j -0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [-0.+0.j -0.+0.j -0.+0.j -0.+0.j -0.+0.j -1.+0.j -0.+0.j -0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  1.+0.j  0

Note that only the former has single $-1$ on the bottom-right corner (`j` stands for the imaginary unit).

## The Grover's diffusion operator

The Grover's diffusion (inversion) operator plays a vital role in the algorithm, as it amplifies the amplitude(s) of the marked element(s) if applied for correct number of times. It can be presented by the following unitary matrix:

$$ I-2 \mymatrix{ccc}{
    \frac{1}{N}  & \cdots & \frac{1}{N} \\ 
    \vdots & \ddots & \vdots \\
    \frac{1}{N}  & \cdots & \frac{1}{N} \\ 
    }  , $$

where $N$ is the dimension of the matrix. The implementation of this operator is known. Here is an example circuit given for four qubits $(N = 16)$.

<img src="..\images/grover_diffusion_operator.png" width="40%" align="center">

In general, one can apply Hadamard and $NOT$ gates to each qubit, and then use a multi-controlled $Z$ gate followed by the same $NOT$ and Hadamard gates. 

Please check the end of the notebook to read about why those gates are used to implement the diffusion operator.

### Task 2 
    
Create a 3-qubit register and design the Grover diffusion operator based on the circuit above. Print the circuit and its matrix form by using `cirq.unitary`. Verify that the circuit by checking the entries of the matrix.

In [12]:
import cirq
from cirq import H, X, Z

#
# your solution
#
qlist = cirq.LineQubit.range(3)
circuit = cirq.Circuit()

hadamard = H.on_each(*qlist)
xgate = X.on_each(*qlist)

circuit.append(hadamard)
circuit.append(xgate)
circuit.append(Z(qlist[2]).controlled_by(qlist[0], qlist[1]))
circuit.append(hadamard)
circuit.append(xgate)

print(cirq.unitary(circuit))

[[ 0.25+0.j  0.25+0.j  0.25+0.j  0.25+0.j  0.25+0.j  0.25+0.j  0.25+0.j
  -0.75+0.j]
 [-0.25+0.j -0.25+0.j -0.25+0.j -0.25+0.j -0.25+0.j -0.25+0.j  0.75+0.j
  -0.25+0.j]
 [-0.25+0.j -0.25+0.j -0.25+0.j -0.25+0.j -0.25+0.j  0.75+0.j -0.25+0.j
  -0.25+0.j]
 [ 0.25+0.j  0.25+0.j  0.25+0.j  0.25+0.j -0.75+0.j  0.25+0.j  0.25+0.j
   0.25+0.j]
 [-0.25+0.j -0.25+0.j -0.25+0.j  0.75+0.j -0.25+0.j -0.25+0.j -0.25+0.j
  -0.25+0.j]
 [ 0.25+0.j  0.25+0.j -0.75+0.j  0.25+0.j  0.25+0.j  0.25+0.j  0.25+0.j
   0.25+0.j]
 [ 0.25+0.j -0.75+0.j  0.25+0.j  0.25+0.j  0.25+0.j  0.25+0.j  0.25+0.j
   0.25+0.j]
 [ 0.75+0.j -0.25+0.j -0.25+0.j -0.25+0.j -0.25+0.j -0.25+0.j -0.25+0.j
  -0.25+0.j]]


[click for our solution](B01_Grover_Algorithm_Revisited_Solutions.ipynb#task2)

## Grover's Algorithm

Finally, we are ready to implement the Grover's Algorithm. Let us consider the Grover's Search for four qubits with proposed simplified oracle. We make an initialization, then apply the oracle (the one we prepared before) and the diffusion operator alternately for $\pi\sqrt{2^4}/4\approx 3$ times. Eventually, the state 1111 should be measured more often than the other basis states. 

### Task 3

Fill the `oracle` and `grover_diffusion` so that they will form a 4-qubit Grover's algorithm. You can use the code from previous tasks. Verify the results by analyzing the statistics of the measurements. Note you can use `yield` for sending consecutive gates. 

In [13]:
import cirq
from cirq import H, Z, X

qq = cirq.LineQubit.range(4)
circuit = cirq.Circuit()
circuit.append(H.on_each(*qq))

def oracle():
    #
    # your solution
    #
    yield Z(qq[3]).controlled_by(*(qq[0:3]))


def grover_diffusion():
    yield H.on_each(*qq)
    yield X.on_each(*qq)
    #
    # your solution
    #
    yield Z(qq[3]).controlled_by(*(qq[0:3]))
    yield H.on_each(*qq)
    yield X.on_each(*qq)

for i in range(3):
    circuit.append(oracle())
    circuit.append(grover_diffusion())
circuit.append(cirq.measure(*qq, key='result'))

# determine the statistics of the measurements
s = cirq.Simulator() 
trials_number = 1000
samples = s.run(circuit, repetitions=trials_number)

def bitstring(bits):
    return "".join(str(int(b)) for b in bits)

counts = samples.histogram(key="result",fold_func=bitstring)

print("Measurement output: ", counts)
print("Probability of measuring 1111: ", counts.get('1111')/trials_number)
print("Probability of not measuring 1111: ", 1- counts.get('1111')/trials_number)

Measurement output:  Counter({'0000': 800, '1111': 182, '1001': 4, '0101': 4, '0110': 3, '1100': 2, '1010': 1, '1101': 1, '0100': 1, '1011': 1, '0011': 1})
Probability of measuring 1111:  0.182
Probability of not measuring 1111:  0.8180000000000001


[click for our solution](B01_Grover_Algorithm_Revisited_Solutions.ipynb#task3)

## Are we done?

We have just implemented the Grover's search algorithm finding 1111 label in $\approx \pi\sqrt{N}/4$ time. In this example, we have implemented a very simple oracle which required using the knowledge about the element we were searching for.

In the next notebooks, we will implement less trivial oracles and use Grover's Algorithm to help us in finding out the solution of a harder problem.

----

<h2>Why these gates are chosen? (Optional) </h2>

Now let's try to understand why these gates are chosen. Let's recall the diffusion operator:

$$ 2 \mymatrix{ccc}{
    \frac{1}{N}  & \cdots & \frac{1}{N} \\ 
    \vdots & \ddots & \vdots \\
    \frac{1}{N}  & \cdots & \frac{1}{N} \\ 
    } 
- I . $$

Recall that the diffusion operator can be expressed as $D=2\ket{u}\bra{u}-I$ where $\ket{u}=H^{\otimes n}\ket{0^n}$ is the equal superposition vector. We will simply denote $\ket{0^n}$ by $\ket{\mathbf{0}}$. 


Instead of $D$, we will implement $-D$. Since the two operators differ by a negative sign, they yield the same observation probabilities. 

For instance, if $D \ket{\psi} = \ket{\phi}$, then $-D \ket{\psi} = -\ket{\phi}$ and the states $\ket{\phi}$ and $-\ket{\phi}$ have identical observation probabilities.

- To start with let's express $-D$ as follows:

\begin{align*}
-D=-(2\ket{u}\bra{u}-I) = I - 2\ket{u}\bra{u} &= I - 2(H^{\otimes n}\ket{\mathbf{0}}\bra{\mathbf{0}}H^{\otimes n}) \\
&= H^{\otimes n}H^{\otimes n} - 2(H^{\otimes n}\ket{\mathbf{0}}\bra{\mathbf{0}}H^{\otimes n})\\
&=H^{\otimes n} (H^{\otimes n} - 2\ket{\mathbf{0}}\bra{\mathbf{0}}H^{\otimes n}) \\
&=H^{\otimes n} (I - 2\ket{\mathbf{0}}\bra{\mathbf{0}})H^{\otimes n}
\end{align*}

<i>Looking at this expression, we understand why we have $H$ gates at the beginning and at the end</i>

- Now let us look at the effect of applying $I - 2\ket{\mathbf{0}}\bra{\mathbf{0}}$ to any arbitrary state.

$(I - 2\ket{\mathbf{0}}\bra{\mathbf{0}}) \ket{x} = \ket{x} - 2\ket{\mathbf{0}}\braket{\mathbf{0}}{x}.$

If $\ket{x}=\ket{\mathbf{0}}$, since $\braket{\mathbf{0}}{\mathbf{0}}=1$, then $ \ket{x} - 2\ket{\mathbf{0}}\braket{\mathbf{0}}{x} = \ket{\mathbf{0}} - 2\ket{\mathbf{0}}\braket{\mathbf{0}}{\mathbf{0}} =\ket{\mathbf{0}} -2\ket{\mathbf{0}} = - \ket{\mathbf{0}}$.

If $\ket{x}\neq \ket{\mathbf{0}}$, since $\braket{\mathbf{0}}{x}=0$, then $ \ket{x} - 2\ket{\mathbf{0}}\braket{\mathbf{0}}{x} =  \ket{x} - 2\ket{\mathbf{0}}\cdot 0  = \ket{x}$.

Hence, the effect of $I - 2\ket{\mathbf{0}}\bra{\mathbf{0}}$ is flipping the amplitude of the state $\ket{\mathbf{0}}$.

- How to check if the quantum state is equal to $\ket{\bf{0}}$ and flip its amplitude?

If all qubits were in state $\ket{1}$, then we could use a multi-controlled $Z$ gate to flip the amplitude. Instead, we need to apply a multi-controlled $Z$ gate when all qubits are in state state $\ket{0}$.

 <i> That's why we first apply $X$ gates and then apply multi-controlled $Z$ gate. Afterwards, we set the qubits back by applying $X$ gates again (Recall the multiple controlled constructions notebook.). </i>